# API Design — Comprehensive Guide

APIs are the **contracts** between services and clients. Well-designed APIs are intuitive, consistent, versioned, documented, and secure. This notebook covers REST, GraphQL, gRPC, API versioning, error handling, rate limiting, security, and documentation best practices.

## Table of Contents
1. [REST API Design Principles](#1-rest-api-design-principles) (Resources, HTTP Methods, Status Codes, HATEOAS)
2. [GraphQL Fundamentals](#2-graphql-fundamentals) (Schema, Queries, Mutations, Subscriptions)
3. [gRPC & Protocol Buffers](#3-grpc--protocol-buffers)
4. [API Versioning Strategies](#4-api-versioning-strategies)
5. [Error Handling & Status Codes](#5-error-handling--status-codes)
6. [Rate Limiting & API Security](#6-rate-limiting--throttling) (Authentication Headers, Rate Limiting)
7. [API Documentation (OpenAPI/Swagger)](#7-api-documentation-openapiswagger)
8. [Pagination, Filtering & Sorting](#8-pagination-filtering--sorting) (Offset, Cursor-based)

---

## 1. REST API Design Principles

REST (Representational State Transfer) is the dominant paradigm for web APIs. It uses **resources** (nouns) and **HTTP methods** (verbs).

### Core Principles

| Principle | Description |
|-----------|-------------|
| **Resources** | Use nouns, not verbs: `/users`, `/orders`, `/products` |
| **HTTP Methods** | GET (read), POST (create), PUT (replace), PATCH (update), DELETE (remove) |
| **Stateless** | Each request contains all information needed; no server-side session |
| **Uniform Interface** | Consistent URL patterns, response formats, and error handling |
| **HATEOAS** | Responses include links to related resources/actions |

### Resource Naming Conventions

```
✅ Good:
GET    /users              # List all users
GET    /users/123          # Get user 123
POST   /users              # Create a user
PUT    /users/123          # Replace user 123
PATCH  /users/123          # Update user 123
DELETE /users/123          # Delete user 123

❌ Bad:
GET    /getUsers
POST   /createUser
POST   /deleteUser/123
```

### Nested Resources

```
GET /users/123/orders           # Orders for user 123
GET /users/123/orders/456       # Order 456 of user 123
POST /users/123/orders          # Create order for user 123
```

### Idempotency

| Method | Idempotent? | Safe? |
|--------|-------------|-------|
| GET | ✅ Yes | ✅ Yes |
| HEAD | ✅ Yes | ✅ Yes |
| OPTIONS | ✅ Yes | ✅ Yes |
| PUT | ✅ Yes | ❌ No |
| DELETE | ✅ Yes | ❌ No |
| POST | ❌ No | ❌ No |
| PATCH | ❌ No | ❌ No |

> **Idempotent**: Multiple identical requests have the same effect as a single request.  
> **Safe**: The request doesn't modify server state.

### HATEOAS (Hypermedia as the Engine of Application State)

HATEOAS enables clients to navigate API resources through hypermedia links embedded in responses.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    HATEOAS Request Flow                             │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│    Client                                    Server                 │
│       │                                         │                   │
│       │  GET /api/orders/123                    │                   │
│       │────────────────────────────────────────>│                   │
│       │                                         │                   │
│       │  200 OK                                 │                   │
│       │  {                                      │                   │
│       │    "id": 123,                           │                   │
│       │    "status": "pending",                 │                   │
│       │    "total": 99.99,                      │                   │
│       │    "_links": {                          │                   │
│       │      "self":   "/api/orders/123",       │                   │
│       │      "pay":    "/api/orders/123/pay",   │                   │
│       │      "cancel": "/api/orders/123/cancel",│                   │
│       │      "items":  "/api/orders/123/items"  │                   │
│       │    }                                    │                   │
│       │  }                                      │                   │
│       │<────────────────────────────────────────│                   │
│       │                                         │                   │
│       │  POST /api/orders/123/pay               │                   │
│       │  (discovered from _links)               │                   │
│       │────────────────────────────────────────>│                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

**HATEOAS Benefits:**
- Decouples clients from URL structure
- Enables discoverability of available actions
- API can evolve without breaking clients

In [ ]:
# Example: FastAPI RESTful API
from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel
from typing import Optional
from datetime import datetime

app = FastAPI(title="User API", version="1.0.0")

# Pydantic models for request/response validation
class UserCreate(BaseModel):
    name: str
    email: str
    role: str = "user"

class UserUpdate(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    role: Optional[str] = None

class UserResponse(BaseModel):
    id: int
    name: str
    email: str
    role: str
    created_at: datetime

# In-memory database simulation
users_db = {}
next_id = 1

@app.get("/v1/users", response_model=list[UserResponse])
def list_users(skip: int = 0, limit: int = 20):
    """List all users with pagination."""
    return list(users_db.values())[skip:skip + limit]

@app.get("/v1/users/{user_id}", response_model=UserResponse)
def get_user(user_id: int):
    """Get a specific user by ID."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    return users_db[user_id]

@app.post("/v1/users", response_model=UserResponse, status_code=status.HTTP_201_CREATED)
def create_user(user: UserCreate):
    """Create a new user."""
    global next_id
    new_user = {
        "id": next_id,
        **user.model_dump(),
        "created_at": datetime.now()
    }
    users_db[next_id] = new_user
    next_id += 1
    return new_user

@app.patch("/v1/users/{user_id}", response_model=UserResponse)
def update_user(user_id: int, user: UserUpdate):
    """Partially update a user."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    stored_user = users_db[user_id]
    update_data = user.model_dump(exclude_unset=True)
    for key, value in update_data.items():
        stored_user[key] = value
    return stored_user

@app.delete("/v1/users/{user_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_user(user_id: int):
    """Delete a user."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    del users_db[user_id]

print("FastAPI REST example defined.")

## 2. GraphQL Fundamentals

GraphQL is a query language for APIs that allows clients to request exactly the data they need.

### REST vs GraphQL

| Aspect | REST | GraphQL |
|--------|------|--------|
| **Endpoints** | Multiple (`/users`, `/posts`) | Single (`/graphql`) |
| **Data fetching** | Fixed response shape | Client specifies fields |
| **Over-fetching** | Common issue | Eliminated |
| **Under-fetching** | Requires multiple requests | Single request for related data |
| **Versioning** | URL or header based | Schema evolution |
| **Caching** | HTTP caching works well | Requires client-side caching |
| **Learning curve** | Lower | Higher |
| **Tooling** | Widespread | Growing rapidly |

### GraphQL Schema Definition

```graphql
type User {
  id: ID!
  name: String!
  email: String!
  posts: [Post!]!
}

type Post {
  id: ID!
  title: String!
  content: String
  author: User!
}

type Query {
  user(id: ID!): User
  users: [User!]!
  post(id: ID!): Post
}

type Mutation {
  createUser(name: String!, email: String!): User!
  createPost(title: String!, content: String, authorId: ID!): Post!
}
```

### GraphQL Query Examples

```graphql
# Get specific fields
query {
  user(id: "123") {
    name
    email
  }
}

# Get nested data in one request
query {
  user(id: "123") {
    name
    posts {
      title
      content
    }
  }
}

# Mutations
mutation {
  createUser(name: "Alice", email: "alice@example.com") {
    id
    name
  }
}
```

In [ ]:
# Example: Strawberry GraphQL with FastAPI
import strawberry
from typing import List, Optional

@strawberry.type
class User:
    id: int
    name: str
    email: str

@strawberry.type
class Post:
    id: int
    title: str
    content: Optional[str]
    author_id: int

# Sample data
users_data = [
    User(id=1, name="Alice", email="alice@example.com"),
    User(id=2, name="Bob", email="bob@example.com"),
]

posts_data = [
    Post(id=1, title="First Post", content="Hello World", author_id=1),
    Post(id=2, title="GraphQL Guide", content="Learn GraphQL", author_id=1),
]

@strawberry.type
class Query:
    @strawberry.field
    def users(self) -> List[User]:
        return users_data
    
    @strawberry.field
    def user(self, id: int) -> Optional[User]:
        return next((u for u in users_data if u.id == id), None)
    
    @strawberry.field
    def posts(self, author_id: Optional[int] = None) -> List[Post]:
        if author_id:
            return [p for p in posts_data if p.author_id == author_id]
        return posts_data

schema = strawberry.Schema(query=Query)

# Execute a query
result = schema.execute_sync("""
    query {
        users {
            id
            name
        }
        posts(authorId: 1) {
            title
        }
    }
""")
print("GraphQL Result:")
print(result.data)

## 3. gRPC & Protocol Buffers

gRPC is a high-performance RPC framework using Protocol Buffers for serialization.

### When to Use gRPC

| Use Case | gRPC | REST |
|----------|------|------|
| Microservices internal | ✅ Preferred | ⚠️ Acceptable |
| Public APIs | ⚠️ Less common | ✅ Preferred |
| High throughput | ✅ Excellent | ⚠️ Good |
| Streaming | ✅ Native support | ⚠️ Workarounds needed |
| Browser clients | ⚠️ gRPC-Web needed | ✅ Native |
| Strong typing | ✅ Built-in | ⚠️ OpenAPI/Pydantic |

### Protocol Buffer Definition

```protobuf
syntax = "proto3";

package user;

service UserService {
  rpc GetUser (GetUserRequest) returns (User);
  rpc ListUsers (ListUsersRequest) returns (stream User);
  rpc CreateUser (CreateUserRequest) returns (User);
}

message User {
  int32 id = 1;
  string name = 2;
  string email = 3;
  Role role = 4;
}

enum Role {
  USER = 0;
  ADMIN = 1;
}

message GetUserRequest {
  int32 id = 1;
}

message ListUsersRequest {
  int32 page = 1;
  int32 limit = 2;
}

message CreateUserRequest {
  string name = 1;
  string email = 2;
}
```

### gRPC Streaming Patterns

```
Unary:           Client ──req──> Server ──resp──> Client

Server Stream:   Client ──req──> Server ══msgs══> Client

Client Stream:   Client ══msgs══> Server ──resp──> Client

Bidirectional:   Client <══msgs══> Server
```

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# API Protocol Comparison
protocols = ['REST/JSON', 'GraphQL', 'gRPC/Protobuf']

# Metrics (relative scale 1-10)
metrics = {
    'Payload Size': [3, 5, 9],  # Lower is better (inverted for display)
    'Serialization Speed': [4, 5, 9],
    'Browser Support': [10, 9, 4],
    'Streaming Support': [3, 5, 10],
    'Type Safety': [5, 8, 10],
    'Tooling Maturity': [10, 8, 7],
}

fig = go.Figure()

categories = list(metrics.keys())
colors = ['#3498db', '#e74c3c', '#2ecc71']

for i, protocol in enumerate(protocols):
    values = [metrics[cat][i] for cat in categories]
    values.append(values[0])  # Close the radar
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories + [categories[0]],
        fill='toself',
        name=protocol,
        line=dict(color=colors[i]),
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 10])),
    title='API Protocol Comparison',
    template='plotly_white',
    showlegend=True
)
fig

## 4. API Versioning Strategies

APIs evolve. Versioning strategies help maintain backward compatibility.

### Versioning Approaches

| Strategy | Example | Pros | Cons |
|----------|---------|------|------|
| **URL Path** | `/v1/users`, `/v2/users` | Clear, easy caching | Multiple codebases |
| **Query Param** | `/users?version=1` | Flexible | Can be forgotten |
| **Header** | `Accept: application/vnd.api+json;version=1` | Clean URLs | Hidden from users |
| **Content Negotiation** | `Accept: application/vnd.company.v1+json` | RESTful | Complex |

### Breaking vs Non-Breaking Changes

| Change Type | Breaking? | Action |
|-------------|-----------|--------|
| Add optional field | ❌ No | Safe to add |
| Add required field | ✅ Yes | New version needed |
| Remove field | ✅ Yes | Deprecate first, then remove |
| Rename field | ✅ Yes | Add alias or new version |
| Change field type | ✅ Yes | New version needed |
| Add new endpoint | ❌ No | Safe to add |
| Change URL structure | ✅ Yes | Redirect old URLs |

### Deprecation Strategy

```http
HTTP/1.1 200 OK
Deprecation: Sat, 1 Jan 2025 00:00:00 GMT
Sunset: Sat, 1 Jul 2025 00:00:00 GMT
Link: </v2/users>; rel="successor-version"
```

## 5. Error Handling & Status Codes

Consistent error responses help clients handle failures gracefully.

### HTTP Status Code Categories

| Range | Category | Common Codes |
|-------|----------|-------------|
| **1xx** | Informational | 100 Continue, 101 Switching Protocols |
| **2xx** | Success | 200 OK, 201 Created, 204 No Content |
| **3xx** | Redirection | 301 Moved, 304 Not Modified |
| **4xx** | Client Error | 400 Bad Request, 401 Unauthorized, 403 Forbidden, 404 Not Found, 422 Unprocessable, 429 Too Many Requests |
| **5xx** | Server Error | 500 Internal Error, 502 Bad Gateway, 503 Service Unavailable, 504 Gateway Timeout |

### Standard Error Response Format (RFC 7807)

```json
{
  "type": "https://api.example.com/errors/validation",
  "title": "Validation Error",
  "status": 422,
  "detail": "The email field must be a valid email address.",
  "instance": "/users/123",
  "errors": [
    {
      "field": "email",
      "message": "Invalid email format",
      "code": "INVALID_FORMAT"
    }
  ],
  "trace_id": "abc-123-def-456"
}
```

In [ ]:
# Status Code Distribution Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Common status codes with usage frequency
status_codes = {
    '200 OK': 70,
    '201 Created': 8,
    '204 No Content': 5,
    '400 Bad Request': 6,
    '401 Unauthorized': 3,
    '403 Forbidden': 1,
    '404 Not Found': 4,
    '422 Validation': 2,
    '500 Server Error': 1,
}

colors = {
    '2xx': '#2ecc71',
    '4xx': '#f39c12',
    '5xx': '#e74c3c'
}

bar_colors = []
for code in status_codes.keys():
    if code.startswith('2'):
        bar_colors.append(colors['2xx'])
    elif code.startswith('4'):
        bar_colors.append(colors['4xx'])
    else:
        bar_colors.append(colors['5xx'])

fig = go.Figure(data=[
    go.Bar(
        x=list(status_codes.keys()),
        y=list(status_codes.values()),
        marker_color=bar_colors,
        text=list(status_codes.values()),
        textposition='auto'
    )
])

fig.update_layout(
    title='Typical API Status Code Distribution (Healthy API)',
    xaxis_title='Status Code',
    yaxis_title='Frequency (%)',
    template='plotly_white'
)
fig

## 6. Rate Limiting & Throttling

Rate limiting protects APIs from abuse and ensures fair usage.

### Common Algorithms

| Algorithm | Description | Pros | Cons |
|-----------|-------------|------|------|
| **Token Bucket** | Tokens added at fixed rate, consumed per request | Allows bursts | Complex to implement |
| **Leaky Bucket** | Requests processed at constant rate | Smooth output | No burst handling |
| **Fixed Window** | Count requests per time window | Simple | Boundary burst problem |
| **Sliding Window** | Rolling window of requests | Accurate | Memory overhead |

### Rate Limit Headers

```http
HTTP/1.1 200 OK
X-RateLimit-Limit: 100
X-RateLimit-Remaining: 95
X-RateLimit-Reset: 1609459200
Retry-After: 60
```

### Rate Limiting by Tier

| Tier | Requests/Hour | Burst Limit | Use Case |
|------|---------------|-------------|----------|
| Free | 100 | 10/min | Developers, testing |
| Basic | 1,000 | 100/min | Small apps |
| Pro | 10,000 | 500/min | Production apps |
| Enterprise | 100,000+ | Custom | High-volume |

### API Security & Authentication

Securing APIs involves multiple layers: authentication, authorization, transport security, and input validation.

```
┌─────────────────────────────────────────────────────────────────────┐
│                    JWT Authentication Flow                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│    Client                    Auth Server              API Server    │
│       │                          │                         │        │
│       │  1. POST /login          │                         │        │
│       │  {email, password}       │                         │        │
│       │─────────────────────────>│                         │        │
│       │                          │                         │        │
│       │  2. 200 OK               │                         │        │
│       │  {access_token,          │                         │        │
│       │   refresh_token,         │                         │        │
│       │   expires_in: 3600}      │                         │        │
│       │<─────────────────────────│                         │        │
│       │                          │                         │        │
│       │  3. GET /api/resource                              │        │
│       │  Authorization: Bearer <access_token>              │        │
│       │───────────────────────────────────────────────────>│        │
│       │                                                    │        │
│       │                          │  4. Validate JWT ───────│        │
│       │                          │     - Check signature   │        │
│       │                          │     - Verify expiration │        │
│       │                          │     - Extract claims    │        │
│       │                                                    │        │
│       │  5. 200 OK {data}                                  │        │
│       │<───────────────────────────────────────────────────│        │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### Authentication Methods Comparison

| Method | Header Format | Use Case | Security |
|--------|--------------|----------|----------|
| **API Key** | `X-API-Key: sk_live_xxx` | Simple integrations | Low |
| **Basic Auth** | `Authorization: Basic base64(user:pass)` | Internal APIs | Low |
| **Bearer Token** | `Authorization: Bearer <jwt>` | Modern APIs | High |
| **OAuth 2.0** | `Authorization: Bearer <oauth_token>` | Third-party access | High |
| **mTLS** | Client certificate | Service-to-service | Very High |

In [ ]:
# API Security Implementation with FastAPI
from fastapi import FastAPI, Depends, HTTPException, status, Header, Request
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials, APIKeyHeader
from pydantic import BaseModel
from datetime import datetime, timedelta
from typing import Optional, Dict
import time
from collections import defaultdict

app = FastAPI(title="Secure API")

# ============================================
# 1. API Key Authentication
# ============================================
API_KEYS = {"sk_live_abc123": "user_1", "sk_live_xyz789": "user_2"}
api_key_header = APIKeyHeader(name="X-API-Key", auto_error=False)

async def verify_api_key(api_key: str = Depends(api_key_header)):
    if api_key not in API_KEYS:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid API Key",
            headers={"WWW-Authenticate": "ApiKey"}
        )
    return API_KEYS[api_key]

# ============================================
# 2. JWT Bearer Token Authentication  
# ============================================
import jwt  # PyJWT library

SECRET_KEY = "your-secret-key-change-in-production"
ALGORITHM = "HS256"
bearer_scheme = HTTPBearer(auto_error=False)

def create_access_token(data: dict, expires_delta: timedelta = timedelta(hours=1)):
    to_encode = data.copy()
    expire = datetime.utcnow() + expires_delta
    to_encode.update({"exp": expire, "iat": datetime.utcnow()})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def decode_token(token: str) -> dict:
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except jwt.ExpiredSignatureError:
        raise HTTPException(status_code=401, detail="Token expired")
    except jwt.InvalidTokenError:
        raise HTTPException(status_code=401, detail="Invalid token")

async def get_current_user(credentials: HTTPAuthorizationCredentials = Depends(bearer_scheme)):
    if not credentials:
        raise HTTPException(status_code=401, detail="Missing authorization header")
    return decode_token(credentials.credentials)

# ============================================
# 3. Rate Limiting Middleware
# ============================================
class RateLimiter:
    def __init__(self, requests_per_minute: int = 60):
        self.rpm = requests_per_minute
        self.requests: Dict[str, list] = defaultdict(list)
    
    def check(self, client_id: str) -> dict:
        now = time.time()
        minute_ago = now - 60
        
        # Remove old requests
        self.requests[client_id] = [
            t for t in self.requests[client_id] if t > minute_ago
        ]
        
        current = len(self.requests[client_id])
        remaining = max(0, self.rpm - current)
        
        headers = {
            "X-RateLimit-Limit": str(self.rpm),
            "X-RateLimit-Remaining": str(remaining),
            "X-RateLimit-Reset": str(int(now + 60))
        }
        
        if current >= self.rpm:
            raise HTTPException(
                status_code=429,
                detail="Rate limit exceeded",
                headers={**headers, "Retry-After": "60"}
            )
        
        self.requests[client_id].append(now)
        return headers

rate_limiter = RateLimiter(requests_per_minute=100)

# ============================================
# Secure Endpoints
# ============================================
@app.post("/auth/login")
async def login(email: str, password: str):
    """Authenticate and receive JWT token."""
    # In production: validate against database
    if email == "admin@example.com" and password == "secret":
        token = create_access_token({"sub": email, "role": "admin"})
        return {
            "access_token": token,
            "token_type": "bearer",
            "expires_in": 3600
        }
    raise HTTPException(status_code=401, detail="Invalid credentials")

@app.get("/api/protected")
async def protected_endpoint(
    request: Request,
    user: dict = Depends(get_current_user)
):
    """JWT-protected endpoint with rate limiting."""
    client_ip = request.client.host if request.client else "unknown"
    rate_headers = rate_limiter.check(client_ip)
    
    return {
        "message": f"Hello, {user.get('sub')}",
        "role": user.get("role"),
        "rate_limit": rate_headers
    }

print("Security endpoints configured:")
print("  POST /auth/login - Get JWT token")
print("  GET /api/protected - JWT + rate limited endpoint")
print("\nExample JWT:", create_access_token({"sub": "demo@example.com"})[:50] + "...")

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Token Bucket Algorithm Visualization
time = np.arange(0, 20, 0.1)
bucket_capacity = 10
refill_rate = 2  # tokens per second

# Simulate token bucket
tokens = []
current_tokens = bucket_capacity
requests = [2, 5, 8, 10, 12, 15, 18]  # Request times
request_costs = [3, 5, 4, 6, 2, 3, 5]  # Tokens per request

request_idx = 0
for t in time:
    # Refill tokens
    current_tokens = min(bucket_capacity, current_tokens + refill_rate * 0.1)
    
    # Check for requests
    if request_idx < len(requests) and t >= requests[request_idx]:
        if current_tokens >= request_costs[request_idx]:
            current_tokens -= request_costs[request_idx]
        request_idx += 1
    
    tokens.append(current_tokens)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=time, y=tokens,
    mode='lines',
    name='Available Tokens',
    line=dict(color='#3498db', width=2)
))

fig.add_hline(y=bucket_capacity, line_dash='dash', line_color='green',
              annotation_text='Max Capacity')

for i, (req_time, cost) in enumerate(zip(requests, request_costs)):
    fig.add_vline(x=req_time, line_dash='dot', line_color='red', opacity=0.5)
    fig.add_annotation(x=req_time, y=bucket_capacity + 1, text=f'Req ({cost})', showarrow=False)

fig.update_layout(
    title='Token Bucket Rate Limiting',
    xaxis_title='Time (seconds)',
    yaxis_title='Tokens Available',
    template='plotly_white',
    yaxis=dict(range=[0, bucket_capacity + 2])
)
fig

## 7. API Documentation (OpenAPI/Swagger)

Good documentation is critical for API adoption.

### OpenAPI Specification (OAS)

```yaml
openapi: 3.0.0
info:
  title: User API
  version: 1.0.0
  description: API for managing users

paths:
  /users:
    get:
      summary: List all users
      parameters:
        - name: limit
          in: query
          schema:
            type: integer
            default: 20
      responses:
        '200':
          description: Successful response
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/User'
    post:
      summary: Create a user
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/UserCreate'
      responses:
        '201':
          description: User created

components:
  schemas:
    User:
      type: object
      properties:
        id:
          type: integer
        name:
          type: string
        email:
          type: string
          format: email
```

### Documentation Best Practices

1. **Include examples** for every endpoint
2. **Document errors** with codes and messages
3. **Show authentication** requirements clearly
4. **Provide SDKs** in popular languages
5. **Interactive playground** (Swagger UI, Redoc)
6. **Changelog** for version updates

## 8. Pagination, Filtering & Sorting

### Pagination Strategies

| Strategy | Example | Pros | Cons |
|----------|---------|------|------|
| **Offset** | `?page=2&limit=20` | Simple, familiar | Inconsistent with inserts/deletes |
| **Cursor** | `?cursor=abc123&limit=20` | Consistent, efficient | Can't jump to page |
| **Keyset** | `?after_id=100&limit=20` | Very efficient | Requires sorted ID |

### Cursor-Based Pagination Response

```json
{
  "data": [...],
  "pagination": {
    "total": 1000,
    "has_next": true,
    "has_prev": true,
    "next_cursor": "eyJpZCI6MTAxfQ==",
    "prev_cursor": "eyJpZCI6ODl9"
  }
}
```

### Filtering & Sorting

```
# Filtering
GET /users?status=active&role=admin
GET /users?created_after=2024-01-01
GET /users?email=*@company.com

# Sorting
GET /users?sort=created_at&order=desc
GET /users?sort=-created_at,+name  # Descending created_at, ascending name

# Field Selection (sparse fieldsets)
GET /users?fields=id,name,email
```

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Pagination Performance Comparison
dataset_sizes = [1000, 10000, 100000, 1000000, 10000000]

# Query time for page 1 (all similar)
offset_page1 = [5, 5, 6, 8, 15]
cursor_page1 = [5, 5, 5, 6, 8]

# Query time for page 1000 (offset degrades)
offset_page1000 = [20, 150, 1500, 15000, 150000]  # O(n) scan
cursor_page1000 = [5, 6, 8, 12, 20]  # O(1) with index

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Page 1 Query Time', 'Page 1000 Query Time'))

# Page 1
fig.add_trace(go.Scatter(x=dataset_sizes, y=offset_page1, name='Offset', 
                         line=dict(color='#e74c3c')), row=1, col=1)
fig.add_trace(go.Scatter(x=dataset_sizes, y=cursor_page1, name='Cursor',
                         line=dict(color='#2ecc71')), row=1, col=1)

# Page 1000
fig.add_trace(go.Scatter(x=dataset_sizes, y=offset_page1000, name='Offset',
                         line=dict(color='#e74c3c'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=dataset_sizes, y=cursor_page1000, name='Cursor',
                         line=dict(color='#2ecc71'), showlegend=False), row=1, col=2)

fig.update_xaxes(type='log', title='Dataset Size')
fig.update_yaxes(type='log', title='Query Time (ms)')
fig.update_layout(
    title='Pagination Strategy Performance',
    template='plotly_white'
)
fig.show()

print("Key Insight: Offset pagination degrades significantly for deep pages.")
print("Use cursor-based pagination for large datasets.")

---

## API Request Lifecycle

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│                          Complete API Request Flow                               │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                  │
│   Client                                                           Server        │
│      │                                                                │          │
│      │  1. HTTP Request                                               │          │
│      │  ┌─────────────────────────────────────────────────────────┐   │          │
│      │  │ POST /api/v1/orders HTTP/1.1                            │   │          │
│      │  │ Host: api.example.com                                   │   │          │
│      │  │ Authorization: Bearer eyJhbGc...                        │   │          │
│      │  │ Content-Type: application/json                          │   │          │
│      │  │ X-Request-ID: req-abc-123                               │   │          │
│      │  │                                                         │   │          │
│      │  │ {"product_id": 42, "quantity": 2}                       │   │          │
│      │  └─────────────────────────────────────────────────────────┘   │          │
│      │────────────────────────────────────────────────────────────────>│          │
│      │                                                                │          │
│      │                    ┌────────────────────────────────────┐      │          │
│      │                    │  2. Server Processing Pipeline     │      │          │
│      │                    ├────────────────────────────────────┤      │          │
│      │                    │  [Rate Limit Check]                │      │          │
│      │                    │         ↓                          │      │          │
│      │                    │  [Authentication]                  │      │          │
│      │                    │         ↓                          │      │          │
│      │                    │  [Authorization]                   │      │          │
│      │                    │         ↓                          │      │          │
│      │                    │  [Input Validation]                │      │          │
│      │                    │         ↓                          │      │          │
│      │                    │  [Business Logic]                  │      │          │
│      │                    │         ↓                          │      │          │
│      │                    │  [Response Serialization]          │      │          │
│      │                    └────────────────────────────────────┘      │          │
│      │                                                                │          │
│      │  3. HTTP Response                                              │          │
│      │  ┌─────────────────────────────────────────────────────────┐   │          │
│      │  │ HTTP/1.1 201 Created                                    │   │          │
│      │  │ Content-Type: application/json                          │   │          │
│      │  │ X-Request-ID: req-abc-123                               │   │          │
│      │  │ X-RateLimit-Remaining: 99                               │   │          │
│      │  │ Location: /api/v1/orders/789                            │   │          │
│      │  │                                                         │   │          │
│      │  │ {                                                       │   │          │
│      │  │   "id": 789,                                            │   │          │
│      │  │   "status": "pending",                                  │   │          │
│      │  │   "_links": {"self": "/orders/789", "pay": "/orders/789/pay"} │        │
│      │  │ }                                                       │   │          │
│      │  └─────────────────────────────────────────────────────────┘   │          │
│      │<────────────────────────────────────────────────────────────────│          │
│                                                                                  │
└─────────────────────────────────────────────────────────────────────────────────┘
```

---

## Key Takeaways

1. **Design for consumers** — APIs should be intuitive and well-documented
2. **Be consistent** — Use standard HTTP methods, status codes, and response formats
3. **Plan for evolution** — Version from day one, deprecate gracefully
4. **Handle errors well** — Provide actionable error messages with proper codes
5. **Protect your API** — Implement authentication, authorization, and rate limiting
6. **Choose the right tool** — REST for simplicity, GraphQL for flexibility, gRPC for performance
7. **Use HATEOAS** — Enable discoverability through hypermedia links
8. **Paginate wisely** — Use cursor-based pagination for large, real-time datasets

## References

- [REST API Design Guidelines](https://restfulapi.net/)
- [OpenAPI Specification](https://swagger.io/specification/)
- [GraphQL Best Practices](https://graphql.org/learn/best-practices/)
- [gRPC Documentation](https://grpc.io/docs/)
- [RFC 7807 - Problem Details for HTTP APIs](https://tools.ietf.org/html/rfc7807)
- [OWASP API Security Top 10](https://owasp.org/www-project-api-security/)